# Env

In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
url = "http://127.0.0.1:8800"

# Mobile Base Control

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
pos = get_mobile_position()
RESULT = {
    "mobile_position": pos
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
result = set_mobile_target_position([-1.0, -1.0, 0])
RESULT = result
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

In [ ]:
def world_to_grid(world_pos, grid_map_shape, grid_size=0.1, floor_pos=[0.0, 0.0]):
    # Grid dimensions from actual grid_map
    height, width = grid_map_shape  # (53, 60)

    # Calculate actual floor bounds
    floor_half_x = (width * grid_size) / 2  # 60 * 0.1 / 2 = 3.0
    floor_half_y = (height * grid_size) / 2  # 53 * 0.1 / 2 = 2.65

    floor_min_x = floor_pos[0] - floor_half_x
    floor_max_y = floor_pos[1] + floor_half_y

    world_x, world_y = world_pos[0], world_pos[1]
    grid_j = int(np.floor((world_x - floor_min_x) / grid_size))
    grid_i = int(np.floor((floor_max_y - world_y) / grid_size))

    # Clamp to grid bounds
    grid_i = max(0, min(height - 1, grid_i))
    grid_j = max(0, min(width - 1, grid_j))

    return grid_i, grid_j

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
grid_map = get_grid_map()
mobile_pos = get_mobile_position()
RESULT = {
    "grid_map": grid_map,
    "mobile_pos": mobile_pos
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
result = response.json()["result"]
grid_map = np.array(result["grid_map"])
mobile_pos = result["mobile_pos"]

robot_grid_i, robot_grid_j = world_to_grid(mobile_pos, grid_map.shape)

grid_map[robot_grid_i, robot_grid_j] = 2
plt.imshow(grid_map, cmap='viridis')
plt.axis('off')
plt.show()

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
grid_map = get_grid_map()
path = plan_mobile_path([1.0, -0.5])
result = follow_mobile_path(path)
RESULT = {
    "grid_map": grid_map,
    "path": path,
    "result": result
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
result = response.json()["result"]
grid_map = np.array(result["grid_map"])
path = result["path"]

print("Path length:", len(path))

# Mark path on grid_map with value 2
for waypoint in path:
    path_grid_i, path_grid_j = world_to_grid(waypoint, grid_map.shape)
    grid_map[path_grid_i, path_grid_j] = 2

plt.imshow(grid_map, cmap='viridis')
plt.axis('off')
plt.show()

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
result = set_mobile_target_position([-1.0, 1.0, PI])
RESULT = result
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
pos = get_mobile_position()
RESULT = {
    "mobile_position": pos
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

# Arm Joint Control

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
pos = get_arm_joint_position()
RESULT = {
    "arm_joint_position": pos
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
arm_joint_position = response.json()["result"]["arm_joint_position"]
arm_joint_position

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
result = set_arm_target_joint([-0.011, -1.032, 0.051, -2.260, 0.063, 1.523, 0.675])
RESULT = result
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

# End Effector Control

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
pos, ori = get_ee_position()
RESULT = {
    "ee_position": {
        "pos": pos,
        "ori": ori
    }
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
ee_position = response.json()["result"]["ee_position"]
ee_position

In [ ]:
ee_target_position = np.array(ee_position["pos"]) + np.array([0.1, 0.0, 0.0 ])
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": f"""
result = set_ee_target_position(({ee_target_position.tolist()}))
RESULT = result
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

# Pick and Place

In [ ]:
# retrieve object positions
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
objects = get_object_positions()
RESULT = {
    "objects": objects
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
objects = response.json()["result"]["objects"]
objects

In [ ]:
# check object position
objects["object_fork_0"]

In [ ]:
# move to object (fork)
target_pos = np.array(objects["object_fork_0"]["pos"][:2])

payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": f"""
grid_map = get_grid_map()
path = plan_mobile_path({target_pos.tolist()})
result = follow_mobile_path(path)
RESULT = {{
    "grid_map": grid_map,
    "path": path,
    "result": result
}}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
result = response.json()["result"]
grid_map = np.array(result["grid_map"])
path = result["path"]

print("Path length:", len(path))

# Mark path on grid_map with value 2
for waypoint in path:
    path_grid_i, path_grid_j = world_to_grid(waypoint, grid_map.shape)
    grid_map[path_grid_i, path_grid_j] = 2

plt.imshow(grid_map, cmap='viridis')
plt.axis('off')
plt.show()

In [ ]:
# pick object(fork)
target_pos = np.array(objects["object_fork_0"]["pos"])

payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": f"""
result = pick_object({target_pos.tolist()}, 0.1, 0.2)
RESULT = result
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

In [ ]:
# move to object (bowl)
target_pos = np.array(objects["object_bowl_0"]["pos"][:2])

payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": f"""
grid_map = get_grid_map()
path = plan_mobile_path({target_pos.tolist()})
result = follow_mobile_path(path)
RESULT = {{
    "grid_map": grid_map,
    "path": path,
    "result": result
}}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
result = response.json()["result"]
grid_map = np.array(result["grid_map"])
path = result["path"]

print("Path length:", len(path))

# Mark path on grid_map with value 2
for waypoint in path:
    path_grid_i, path_grid_j = world_to_grid(waypoint, grid_map.shape)
    grid_map[path_grid_i, path_grid_j] = 2

plt.imshow(grid_map, cmap='viridis')
plt.axis('off')
plt.show()

In [ ]:
# place object(fork) at bowl
target_pos = np.array(objects["object_bowl_0"]["pos"])

payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": f"""
result = place_object({target_pos.tolist()}, 0.2, 0.3)
RESULT = result
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

# Scenario 01 (Cleaning Table)

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
objects = get_object_positions()
place = None
targets = []
for key, obj in objects.items():
    if 'bowl' in key:
        place = (key, obj)
    else:
        targets.append((key, obj))

for target in targets:
    path = plan_mobile_path(target[1]['pos'])
    result = follow_mobile_path(path)
    result = pick_object(target[1]['pos'], 0.1, 0.2)
    path = plan_mobile_path(place[1]['pos'])
    result = follow_mobile_path(path)
    result = place_object(place[1]['pos'], 0.1, 0.2)

RESULT = {
    "objects": objects,
    "place": place,
    "targets": targets
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
objects = response.json()["result"]
objects

# Scenario 02 (Moving)

In [ ]:
# 1. Square Path (1m):
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
print("Starting square path...")
pos = [0, 0, 0]
set_mobile_target_position(pos)
pos[1] -= 1
set_mobile_target_position(pos)
print("1/4 complete")
pos[0] -= 1
set_mobile_target_position(pos)
print("2/4 complete")
pos[1] += 1
set_mobile_target_position(pos)
print("3/4 complete")
pos[0] += 1
set_mobile_target_position(pos)
print("Square complete!")
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
result = response.json()
result

In [ ]:
# 2. Circle Path (radius 0.5m):
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
print("Starting circular motion...")
radius = 0.5
steps = 16
pos = [0, 0, 0]
set_mobile_target_position(pos)
for i in range(steps + 1):
    angle = 2 * PI * i / steps
    x = radius * (1 - math.cos(angle))
    y = radius * math.sin(angle)
    theta = angle
    set_mobile_target_position([x, y, theta])  # Pass as list!
    if i % 4 == 0:
        print(f"Progress: {i*100//steps}%")
print("Circle complete!")
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
result = response.json()
result

In [ ]:
# 3. Rotate in Place:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
print("Rotating in place...")
pos = get_mobile_position()
for i in range(8):
    pos[2] = 2 * PI * (i + 1) / 8  # Update theta only
    set_mobile_target_position(pos)  # Pass list!
print("Rotation complete!")
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
result = response.json()
result

In [ ]:
# 4. Triangle Path:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
print("Starting triangle path...")
pos = [0, 0, 0]
set_mobile_target_position(pos)
size = 1.0
# Point 1
pos = [size, 0, 0]
set_mobile_target_position(pos)
# Point 2
pos = [size / 2, size*math.sqrt(3)/2, 2*PI/3]
set_mobile_target_position(pos)
# Return to origin
pos = [0, 0, -2*PI/3]
set_mobile_target_position(pos)
# Theta reset
pos = [0, 0, 0]
set_mobile_target_position(pos)
print("Triangle complete!")
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
result = response.json()
result

In [ ]:
# 5. Long Distance Movement
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
print("Moving long distance...")
# For distances >3m, increase timeout proportionally
# Example: 5m distance needs ~25 second timeout
set_mobile_target_position([5.0, 0.0, 0], timeout=25.0)
set_mobile_target_position([0.0, 0.0, 0], timeout=25.0)
print("Long distance movement complete!")
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
result = response.json()
result